## Routing

In this the LLM decides which route/path he wants to choose. 

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

import os 

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = "langchain-tutorial"
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [2]:
# There are 2 types of routing. 
# Semantic and logical rounting

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.pydantic_v1 import BaseModel, Field

from typing import Literal

# Data Model
class RouteQuery(BaseModel):
    "Route a user query to the most relevant datasource. Class that will have relevant parameters. For example this datasource is for which language documentation shoud I go with."

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ..., 
        description="Given a user question choose which datasource would be the most relevant for answering their question"
    ) 

# LLM with function call
llm = ChatGroq(temperature=0)

structured_llm = llm.with_structured_output(RouteQuery)

system_prompt = """You are an expert at routing a user question to the appropriate data source. 

Based on the programming language the question si reffering, to route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt), 
        ("human", "{question}"),
    ]
)

router = prompt | structured_llm

In [5]:
question = """Why is the following code not working:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "Speak in {language}"])

prompt.invoke("french")
"""

result = router.invoke(
    {"question": question}
)

result

RouteQuery(datasource='python_docs')

In [6]:
result.datasource

'python_docs'

In [8]:
from langchain_core.runnables import RunnableLambda

def choose_route(result):
    if "python_docs" in result.datasource.lower():
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        return "chain for js_docs"
    else:
        return "golang_docs"
    
full_chain = router | RunnableLambda(choose_route)

In [9]:
full_chain.invoke({"question": question})

'chain for python_docs'

## Query Construction

In [11]:
!pip install youtube-transcript-api

   ---------------------------------------- 0.0/622.3 kB ? eta -:--:--
   ---------------------------------------- 622.3/622.3 kB 4.6 MB/s eta 0:00:00


In [13]:
!pip install pytube

In [18]:
from langchain_community.document_loaders import YoutubeLoader

docs = YoutubeLoader.from_youtube_url(
    "https://youtu.be/pmxfvy3SImg?si=2yEmIkbfs4KAA-JK", 
    add_video_info = True
).load()

docs

PytubeError: Exception while accessing title of https://youtube.com/watch?v=pmxfvy3SImg. Please file a bug report at https://github.com/pytube/pytube